In [1]:
import re
import statistics
import pandas as pd
import numpy as np

from sklearn import model_selection

import keras
from keras import layers
from keras.callbacks import EarlyStopping

In [2]:
total_tweets = 4887
mechanism_id = {
        'absurd': 1,
        'analogy': 2,
        'embarrassment': 3,
        'exaggeration': 4,
        'insults': 5,
        'irony': 6,
        'misunderstanding': 7,
        'parody': 8,
        'reference': 9,
        'stereotype': 10,
        'unmasking': 11,
        'wordplay': 12
    }

model_functions = ['elu', 'relu', 'selu','sigmoid', 'tanh']
layer_units = [15, 30, 50, 75, 100]
epoch = 150

unwanted_chars = ['!', ',', '"', '-', '...','–','XD', 'xD', '¿', '?', '—', '\n', "#", '¡', ':', "“", '.', '(', ')']
unwanted_chars.extend(["¬¬", "\('.')/", "*", '\n', '»', '\x97', '\x85'])

In [3]:
tweets = []
mechanisms = []
words_per_tweet = []

In [4]:
train_data = pd.read_csv("sources/haha_mechanism_target_train.csv")

train_data.drop("id", axis=1, inplace=True)
train_data.drop("target", axis=1, inplace=True)

In [5]:
def sanitize_tweet(tweet):
    for char in unwanted_chars:
        tweet = tweet.replace(char, ' ')
    tweet = re.sub('@\w*', '', tweet) #remove user references
    tweet = re.sub('\$', '$ ', tweet) #split prices chars
    tweet = tweet.split(" ")
    tweet = [token for token in tweet if token != ''] #remove unwanted spaces
    return tweet

In [6]:
global tweets, mechanisms
for idx, row in train_data.iterrows():
    mechanisms.append(mechanism_id.get(train_data.loc[idx, "mechanism"]))
    
    sanitized_tweet = sanitize_tweet(train_data.loc[idx, "text"])
    tweets.append(sanitized_tweet)

In [7]:
num_words_list = list(map(lambda x: len(x), tweets))
median_words_tweet = statistics.median(num_words_list)

del train_data

In [8]:
for i in range (0,total_tweets):
    reshaped_tweet = tweets.pop()
    words_counter = len(reshaped_tweet)

    if  words_counter < median_words_tweet:
        print("add padding")
    elif words_counter > median_words_tweet:
        removed_words = words_counter - median_words_tweet + 1
        reshaped_tweet = reshaped_tweet[4:]

    tweets.append(reshaped_tweet)        

add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add 

add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add padding
add 

In [10]:
#------------------------------------------------------------------------------------------------------

In [ ]:
#X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

In [11]:
my_callbacks = [
    EarlyStopping(monitor="val_loss", patience=PATIENCE, verbose=0, mode="auto", restore_best_weights= True)
]

NameError: name 'PATIENCE' is not defined

In [ ]:
for activation_function in model_functions:
    for recurrent_function in model_functions:
        for recursive_layers_architecture in recursive_layers_architectures:
            gru_gru_model = keras.Sequential(
                [
                    layers.Embedding(input_dim = embedding_matix.shape[0], output_dim = embedding_matrix.shape[1],
                                 input_length = 1, weights = [], trainable = False, mask_zero = True),
                    layers.GRU(units = 1, dropout = DROPOUT, recurrent_dropout = RECURRENT_DROPOUT,
                           kernel_initializer=KERNEL_INITIALIZER1, activation=activation_function, return_sequences=True),
                    layers.GRU(units = 1, dropout = DROPOUT, recurrent_dropout = RECURRENT_DROPOUT,
                           kernel_initializer=KERNEL_INITIALIZER1, activation=activation_function),
                    layers.Dense(TARGETS, activation=activation_function)
                ]
            )

In [ ]:
for activation_function in model_functions:
    for recurrent_function in model_functions:
        for recursive_layers_architecture in recursive_layers_architectures:
            gru_lstm_model = keras.Sequential(
                [
                    layers.Embedding(input_dim = embedding_matix.shape[0], output_dim = embedding_matrix.shape[1],
                                 input_length = 1, weights = [], trainable = False, mask_zero = True),
                    layers.GRU(units = 1, dropout = DROPOUT, recurrent_dropout = RECURRENT_DROPOUT,
                           kernel_initializer=KERNEL_INITIALIZER1, activation=activation_function, return_sequences=True),
                    layers.LSTM(units = 1, dropout = DROPOUT, recurrent_dropout = RECURRENT_DROPOUT,
                           kernel_initializer=KERNEL_INITIALIZER1, activation=activation_function),
                    layers.Dense(TARGETS, activation=activation_function)
                ]
            )

In [ ]:
for activation_function in model_functions:
    for recurrent_function in model_functions:
        for recursive_layers_architecture in recursive_layers_architectures:
            lstm_gru_model = keras.Sequential(
                [
                    layers.Embedding(input_dim = embedding_matix.shape[0], output_dim = embedding_matrix.shape[1],
                                 input_length = 1, weights = [], trainable = False, mask_zero = True),
                    layers.LSTM(units = 1, dropout = DROPOUT, recurrent_dropout = RECURRENT_DROPOUT,
                           kernel_initializer=KERNEL_INITIALIZER1, activation=activation_function, return_sequences=True),
                    layers.GRU(units = 1, dropout = DROPOUT, recurrent_dropout = RECURRENT_DROPOUT,
                           kernel_initializer=KERNEL_INITIALIZER1, activation=activation_function),
                    layers.Dense(TARGETS, activation=activation_function)
                ]
            )

In [ ]:
for activation_function in model_functions:
    for recurrent_function in model_functions:
        for recursive_layers_architecture in recursive_layers_architectures:
            lstm_lstm_model = keras.Sequential(
                [
                    layers.Embedding(input_dim = embedding_matix.shape[0], output_dim = embedding_matrix.shape[1],
                                 input_length = 1, weights = [], trainable = False, mask_zero = True),
                    layers.LSTM(units = 1, dropout = DROPOUT, recurrent_dropout = RECURRENT_DROPOUT,
                           kernel_initializer=KERNEL_INITIALIZER1, activation=activation_function, return_sequences=True),
                    layers.LSTM(units = 1, dropout = DROPOUT, recurrent_dropout = RECURRENT_DROPOUT,
                           kernel_initializer=KERNEL_INITIALIZER1, activation=activation_function),
                    layers.Dense(TARGETS, activation=activation_function)
                ]
            )